In [ ]:
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
from lab1 import PIDSimulation, DELTA, show_value, get_exercise_params, plot_and_visualize, plot, visualize

class PID:
    def __init__(self, kp: float, kd: float, ki: float):
        self.kp = kp
        self.kd = kd
        self.ki = ki
        self.last_x = math.nan
        self.integral = 0

    def control(self, delta: float, x: float) -> float:
        vx = (x - self.last_x) / delta
        self.last_x = x
        self.integral = min(max(self.integral + x * self.ki * delta, -100), 100) # TODO: Better anti-windup
        show_value('integral', self.integral)
        return -x * self.kp - vx * self.kd - self.integral

# TODO: Move this into exercise1?
# TODO: Get rid of explicit references to DELTA?
def simulate(pid: PID, params, max_time: int):
    sim = PIDSimulation(copy.deepcopy(pid), params)
    stable_time = 0
    while sim.t < max_time:
        sim.tick(DELTA)
        if abs(sim.vx) <= 0.01 and abs(sim.x) <= 0.01:
            stable_time += DELTA
            if stable_time > 1.0:
                return sim.t - stable_time
        else:
            stable_time = 0
    return math.inf

In [ ]:
x_axis = np.arange(0, 10, 0.01)
plt.figure('PID stabilization time')
plt.plot(x_axis, [simulate(PID(10, 5, x), params=get_exercise_params(2), max_time=30) for x in x_axis])
plt.ylim(bottom=0)
plt.show()

In [ ]:
import scipy.optimize

def optimize(params):
    func = lambda x: simulate(PID(*x), params, max_time=30)
    bounds = [(0, 10), (0, 10), (0, 10)]
    return scipy.optimize.differential_evolution(func, bounds=bounds)

result = optimize(params=get_exercise_params(3))
result

In [ ]:
# %matplotlib inline
plot_and_visualize(PID(10, 5, 7), get_exercise_params(2))

In [ ]:
from ipywidgets import FloatSlider, Layout

slider_layout = Layout(width='500px')
kp=FloatSlider(1, min=0, max=10, layout=slider_layout)
kd=FloatSlider(0, min=0, max=10, layout=slider_layout)
ki=FloatSlider(0, min=0, max=10, layout=slider_layout)

In [ ]:
%matplotlib widget

from ipywidgets import interact

# Figure that will be reused to plot the interactive graph
figure = plt.figure('Interactive PID')

params = get_exercise_params(3)
def visualize_pid(kp, kd, ki):
    pid = PID(kp, kd, ki)
    # print(simulate(pid, params, max_time=30))
    plot(pid, params, max_time=30, interactive_figure=figure)

interact(visualize_pid, kp=kp,kd=kd, ki=ki)
plt.show()
print(plt.get_fignums())